In [71]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [72]:
def keep(matriz):
    num_paradas = 4
    # Condicion pimera :  el número de paradas es fijo e igual a num_paradas
    if np.sum(matriz) != num_paradas:
        print('Condicion pimera :  el número de paradas es fijo e igual a num_paradas')
        return False
    # Condicion tercera : el viaje tiene que ser un camino cerrado
    elif np.trace(np.linalg.matrix_power(matriz, num_paradas)) <=1:
        print('Condicion tercera : el viaje tiene que ser un camino cerrado')
        return False
    # Condicion cuarta : no puedo ir de una parada a esa misma parada
    elif np.all(np.diag(matriz) != 0):
        print('Condicion cuarta : no puedo ir de una parada a esa misma parada')
        return False
    # Condicion quinta : el grado de (entrada,salida) tiene que ser como mucho (1,1)
    elif np.all(np.sum(matriz, axis=1) > 1):
        print('Condicion quinta : el grado de (entrada,salida) tiene que ser como mucho (1,1)')
        return False
    elif np.all(np.sum(matriz, axis=0) > 1):
        print('Condicion quinta : el grado de (entrada,salida) tiene que ser como mucho (1,1)')
        return False
    else:
        return True

In [73]:
def keep_matrices(lista_matrices):
    keep_ = False
    suma_columnas = 0
    matriz_suma = np.zeros_like(lista_matrices[0]) 
    for A in lista_matrices:
        keep_ = keep(A)
        # Condicion segunda : todas las paradas se usan al menos una vez
        suma_columnas += np.sum(A, axis=0)
        matriz_suma += A
    # Condicion segunda : todas las paradas se usan al menos una vez
    if np.all(suma_columnas < 1):
        print('Condicion segunda : todas las paradas se usan al menos una vez')
        keep_ = False
    # condicion sexta : no hay componentes disconexas
    elif np.any(np.linalg.matrix_power(matriz_suma, matriz_suma.shape[0]-1)) <= 0:
        print('condicion sexta : no hay componentes disconexas')
        keep_ = False

    return keep_

    

In [74]:
lista_matrices = [
    np.array([[0,1,0,0],
              [1,0,0,0],
              [0,0,0,1],
              [0,0,1,0]]
    )

]

In [75]:
print(keep_matrices(lista_matrices))

True


In [76]:
def generar_grafo_aleatorio(num_nodos, num=200000):
    lista_grafos = []
    lista_matrices = []
    for _ in range(num):
        # Crear una matriz de adyacencia vacía (de tamaño num_nodos x num_nodos)
        matriz_adyacencia = np.zeros((num_nodos, num_nodos))

        # Aplicar condiciones sobre la matriz de adyacencia
        for i in range(num_nodos):
            for j in range(num_nodos):
                # Probabilidad de generar una arista de i a j
                matriz_adyacencia[i, j] = random.choice([0, 1])
        if keep(matriz_adyacencia):
            lista_matrices.append(matriz_adyacencia)
            # Crear un grafo dirigido usando la matriz de adyacencia
            G = nx.DiGraph()

            for i in range(num_nodos):
                for j in range(num_nodos):
                    if matriz_adyacencia[i, j] != 0:
                        G.add_edge(i, j)
            
            lista_grafos.append(G)

    return lista_grafos, lista_matrices


In [77]:
import tkinter as tk
import numpy as np

class GrafoInteractivo:
    def __init__(self, num_nodos):
        self.num_nodos = num_nodos
        self.matriz_adyacencia = np.zeros((num_nodos, num_nodos), dtype=int)
        self.nodos = []
        self.aristas = []
        self.nodo_seleccionado = None

        # Crear ventana principal
        self.ventana = tk.Tk()
        self.ventana.title("Crear grafo dirigido")

        # Crear canvas para el grafo
        self.canvas = tk.Canvas(self.ventana, width=600, height=600, bg="white")
        self.canvas.pack()

        # Dibujar nodos
        self.dibujar_nodos()

        # Eventos
        self.canvas.bind("<Button-1>", self.seleccionar_nodo)
        self.ventana.protocol("WM_DELETE_WINDOW", self.salir)

        # Botón para mostrar la matriz de adyacencia
        self.boton_matriz = tk.Button(self.ventana, text="Mostrar matriz de adyacencia", command=self.mostrar_matriz)
        self.boton_matriz.pack()

        # Iniciar loop de la ventana
        self.ventana.mainloop()

    def dibujar_nodos(self):
        # Dibujar nodos de forma circular
        centro_x, centro_y = 300, 300
        radio = 200
        for i in range(self.num_nodos):
            angulo = 2 * np.pi * i / self.num_nodos
            x = centro_x + radio * np.cos(angulo)
            y = centro_y + radio * np.sin(angulo)
            self.nodos.append((x, y))
            self.canvas.create_oval(x - 15, y - 15, x + 15, y + 15, fill="lightblue", outline="black", width=2)
            self.canvas.create_text(x, y, text=str(i), font=("Arial", 12))

    def seleccionar_nodo(self, event):
        # Determinar si se ha hecho clic en un nodo
        for i, (x, y) in enumerate(self.nodos):
            if (x - event.x) ** 2 + (y - event.y) ** 2 <= 15 ** 2:
                if self.nodo_seleccionado is None:
                    # Seleccionar el primer nodo
                    self.nodo_seleccionado = i
                    self.canvas.itemconfig(self.canvas.find_withtag("current"), fill="yellow")
                else:
                    # Crear arista dirigida entre el nodo seleccionado y el clicado
                    nodo_origen = self.nodo_seleccionado
                    nodo_destino = i
                    self.matriz_adyacencia[nodo_origen, nodo_destino] = 1
                    self.aristas.append((nodo_origen, nodo_destino))

                    # Dibujar la arista
                    x1, y1 = self.nodos[nodo_origen]
                    x2, y2 = self.nodos[nodo_destino]
                    self.dibujar_arista(x1, y1, x2, y2)

                    # Deseleccionar el nodo seleccionado
                    self.nodo_seleccionado = None
                    self.canvas.itemconfig(self.canvas.find_withtag("current"), fill="lightblue")
                break

    def dibujar_arista(self, x1, y1, x2, y2):
        # Dibujar línea con flecha entre dos nodos
        self.canvas.create_line(x1, y1, x2, y2, arrow=tk.LAST, width=2, fill="black")

    def mostrar_matriz(self):
        # Imprimir matriz de adyacencia
        print("Matriz de adyacencia:")
        print(self.matriz_adyacencia)

    def salir(self):
        self.ventana.destroy()

# Solicitar al usuario el número de nodos
num_nodos = int(input("Ingrese el número de nodos: "))
GrafoInteractivo(num_nodos)

ModuleNotFoundError: No module named '_tkinter'